In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn #remove unnecessary warnings

In [2]:
data = pd.read_csv('biddings.csv')
dfo = pd.DataFrame(data)


In [3]:
dfo = dfo.sort_values(by=['convert'])
dfo = dfo.reset_index(drop=True)
df1 = dfo.drop(dfo.index[0:996800])


In [4]:
df = df1.reset_index(drop=True) #reset index

#Convert column = shown ad was clicked: 1 = clicked ad
print(df.convert.value_counts())

df.tail()

1    1908
0    1292
Name: convert, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,convert
3195,0.05,-2.45,-4.96,-0.22,-0.82,2.85,3.29,-0.29,-0.39,0.72,...,0.21,-0.19,-0.68,0.03,-0.29,-0.06,0.14,0.14,-0.02,1
3196,0.04,-7.19,-3.74,-0.71,0.36,0.38,0.82,1.37,-0.48,2.45,...,-0.04,0.62,-0.02,-0.38,0.52,-0.10,-0.29,-0.65,0.23,1
3197,-0.01,0.71,2.30,-0.23,-0.67,-0.05,0.24,0.11,0.43,0.55,...,-0.19,0.39,1.06,0.05,-0.87,-0.01,1.79,-0.76,0.22,1
3198,-0.02,2.91,-3.57,-0.11,-0.70,2.69,2.67,-1.02,-0.14,-0.86,...,-0.17,0.89,0.49,0.01,1.17,0.10,0.11,-0.08,0.03,1
3199,-0.01,-0.76,-4.40,-0.19,-0.76,2.45,0.92,-0.50,-0.18,0.01,...,0.01,-1.22,0.24,0.22,0.31,0.10,0.13,-0.17,0.08,1


In [22]:
from sklearn.model_selection import train_test_split

x = df.drop(['convert'],axis=1) #training features
y = df.convert #target

#Set our train/test values
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                  test_size = .10)
#Test sample
y_test.value_counts()

1    196
0    124
Name: convert, dtype: int64

In [25]:
#First model using our undersampled data
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

clf = LinearSVC(random_state=42)
clf.fit(x_train, y_train) 

print('Training accuracy LinearSVC is: %0.2f' % (clf.score(x_train, y_train)))
print('Test accuracy LinearSVC is: %0.2f' % (clf.score(x_test, y_test)))

scores = cross_val_score(clf, x_train, y_train, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

y_pred = clf.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

print('\nThis LinearSVC runs at a speed of:')
%timeit clf.predict(df.iloc[5:6, :88])

Training accuracy LinearSVC is: 0.67
Test accuracy LinearSVC is: 0.67
Cross Val LinearSVC Accuracy: 0.63 (+/- 0.04)

              precision    recall  f1-score   support

           0       0.59      0.48      0.53       124
           1       0.70      0.79      0.75       196

   micro avg       0.67      0.67      0.67       320
   macro avg       0.65      0.63      0.64       320
weighted avg       0.66      0.67      0.66       320


This LinearSVC runs at a speed of:
883 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
from sklearn.metrics import f1_score

#Show f1 score 
print("LinearSVC F1 Score: %0.2f \n" % f1_score(y_test, y_pred))


LinearSVC F1 Score: 0.75 

